In [1]:
import nltk
from nltk.corpus import movie_reviews

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv("/content/drive/MyDrive/BoB/daum_movie_review.csv")

In [5]:
df

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워
...,...,...,...,...
14720,어른들을 위한 동화 정말 오랜만에 좋은 애니를 보았습니다 가족의 소중...,10,2018.01.12,코코
14721,디즈니는 못해도 본전은 한다.,7,2018.01.12,코코
14722,가족을 위한 영화... 괜찮은 영화.~~~,8,2018.01.12,코코
14723,간만에 제대로 잘짜여진 각본의 영화를 봤네 여운이 아직도 남아~어른을 위한 애니~,10,2018.01.12,코코


In [6]:
y = [0 if rate < 6 else 1 for rate in df.rating]

In [7]:
X_train_val, X_test, y_train_val, y_test = train_test_split(df.review.tolist(), y, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42)

# BERT

In [8]:
! pip install datasets

In [8]:
import torch
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

class OurDataset(torch.utils.data.Dataset):
  def __init__(self, inputs, labels):
    self.inputs = inputs
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)



<ipython-input-8-13db67272b57>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [21]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:00:00


In [9]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [28]:
! pip install accelerate -U
! pip install transformers[torch]

In [10]:
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [13]:



# tokenizing
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors ="pt")
val_input = tokenizer(X_val, truncation=True, padding =True, return_tensors="pt")
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")


# Dataset
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")


training_args = TrainingArguments(
  output_dir = './results',
  num_train_epochs =2,
  evaluation_strategy="steps",
  eval_steps = 500,
  per_device_train_batch_size =8,
  per_device_eval_batch_size = 16,
  warmup_steps=200,
  weight_decay=0.01,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics,
)

#fine tuning
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-13db67272b57>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Step,Training Loss,Validation Loss,Accuracy
500,0.568400,0.552210,0.762043
1000,0.568300,0.549447,0.762043
1500,0.559200,0.548704,0.762043
2000,0.559100,0.544501,0.762043


<ipython-input-12-13db67272b57>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
<ipython-input-12-13db67272b57>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
<ipython-input-12-13db67272b57>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
<ipython-input-12-13db67272b57>:17: UserWarning: To copy construct from a tenso

TrainOutput(global_step=2072, training_loss=0.5632340751559578, metrics={'train_runtime': 1289.7641, 'train_samples_per_second': 12.843, 'train_steps_per_second': 1.606, 'total_flos': 2638736663095200.0, 'train_loss': 0.5632340751559578, 'epoch': 2.0})

In [14]:
trainer.evaluate(eval_dataset=test_dataset)

<ipython-input-12-13db67272b57>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


{'eval_loss': 0.5312923789024353,
 'eval_accuracy': 0.7609994568169474,
 'eval_runtime': 73.0875,
 'eval_samples_per_second': 50.378,
 'eval_steps_per_second': 3.161,
 'epoch': 2.0}

# Ko-Bert

In [15]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00


In [12]:
pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-nxhwzokm/kobert-tokenizer_cd5d8d7c05784369a2f09c9c0fb7cfda
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-nxhwzokm/kobert-tokenizer_cd5d8d7c05784369a2f09c9c0fb7cfda
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [11]:
del model
del trainer
torch.cuda.empty_cache()

NameError: ignored

In [13]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [21]:
from transformers import BertModel
from torch.utils.data import DataLoader

# tokenizing
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors ="pt")
val_input = tokenizer(X_val, truncation=True, padding =True, return_tensors="pt")
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")


# Dataset
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)

# Create Dataloader
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# load kobert
bert_model = BertModel.from_pretrained('skt/kobert-base-v1')

# nn model with bert
class MyModel(torch.nn.Module):
  def __init__(self, pretrained_model, token_size, num_labels):
    super(MyModel, self).__init__()
    self.token_size = token_size
    self.num_labels = num_labels
    self.pretrained_model = pretrained_model
    self.classifier = torch.nn.Linear(self.token_size, self.num_labels)

  def forward(self, inputs):
    outputs = self.pretrained_model(**inputs)

    bert_clf_token = outputs.last_hidden_state[:,0,:]

    return self.classifier(bert_clf_token)

model = MyModel(bert_model, num_labels=2, token_size=bert_model.config.hidden_size)


In [15]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F
import time

In [23]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

optim = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 2
total_training_steps = num_epochs * len(train_loader)

scheduler = get_linear_schedule_with_warmup(optimizer=optim,
                                            num_training_steps=total_training_steps,
                                            num_warmup_steps=200)
start = time.time()
train_loss = 0
eval_steps = 500
step = 0

for epoch in range(num_epochs):
  for batch in train_loader:
    model.train()
    optim.zero_grad()

    inputs = {k: v.to(device) for k,v in batch.items() if k != 'labels'}
    labels = batch['labels'].to(device)
    outputs = model(inputs)

    loss = criterion(outputs, F.one_hot(labels, num_classes=2).float())

    train_loss += loss
    loss.backward()
    optim.step()
    scheduler.step()

    step += 1
    if step % eval_steps == 0:
      with torch.no_grad():
        val_loss = 0
        model.eval()
        for batch in val_loader:
          inputs = {k:v.to(device) for k,v in batch.items() if k != 'labels'}
          labels = batch['labels'].to(device)
          outputs=model(inputs)

          loss = criterion(outputs, F.one_hot(labels, num_classes=2).float())
          val_loss += loss
        avg_val_loss = val_loss / len(val_loader)
      avg_train_loss = train_loss / eval_steps
      elapsed = time.time() - start
      print(
          'Step %d, elapsed time: %.2f, train loss:%.4f, validation loss: %.4f' %(step, elapsed, avg_train_loss, avg_val_loss)
      )

<ipython-input-8-13db67272b57>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Step 500, elapsed time: 249.42, train loss:0.4624, validation loss: 0.3513
Step 1000, elapsed time: 501.87, train loss:0.8255, validation loss: 0.3137
Step 1500, elapsed time: 754.06, train loss:1.0767, validation loss: 0.3296
Step 2000, elapsed time: 1006.14, train loss:1.3089, validation loss: 0.3120


In [31]:
from datasets import load_metric

all_predictions = []
metric = load_metric("accuracy")
model.eval()
for batch in test_loader:
  inputs = {k: v.to(device) for k,v in batch.items() if k != 'labels'}
  labels = batch['labels'].to(device)

  with torch.no_grad():
    outputs = model(inputs)

  predictions = torch.argmax(outputs, dim=-1)
  metric.add_batch(predictions=predictions, references=labels)

  all_predictions.extend(predictions.cpu().numpy())

print("Predictions:", all_predictions)

metric.compute()

<ipython-input-8-13db67272b57>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Predictions: [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,

{'accuracy': 0.8734383487235198}

In [28]:
outputs

tensor([[-1.8736,  1.6535],
        [-2.5826,  2.1086]], device='cuda:0')